<a href="https://colab.research.google.com/github/AlcidesChewe/TP_MachineLearning/blob/main/TP_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install numpy-stl

# **Simulacion de rotura**

In [10]:
import numpy as np
from stl import mesh

# Load the mesh using numpy-stl
your_mesh = mesh.Mesh.from_file('beker1.stl')

# Determine the z-coordinates of all vertices
z_coords = your_mesh.vectors[:,:,2]

# Find the median z-coordinate to split the taza approximately in half
median_z = np.median(z_coords)

# Remove triangles whose three vertices are below the median z-coordinate
mask = np.all(z_coords < median_z, axis=1)
your_mesh.vectors[mask] = np.zeros((3, 3))

# Save the modified mesh to a new file
your_mesh.save('beker1_roto.stl')



# **Parte rota**


In [11]:
import numpy as np
from stl import mesh

# Load the mesh using numpy-stl
your_mesh = mesh.Mesh.from_file('beker1.stl')

# Determine the z-coordinates of all vertices
z_coords = your_mesh.vectors[:,:,2]

# Find the median z-coordinate to split the taza approximately in half
median_z = np.median(z_coords)

# Remove triangles whose three vertices are above the median z-coordinate
mask = np.all(z_coords > median_z, axis=1)
your_mesh.vectors[mask] = np.zeros((3, 3))

# Save the modified mesh to a new file
your_mesh.save('beker1_parte_superior.stl')
